In [2]:
import pandas as pd

### Ex1：公司员工数据集
现有一份公司员工数据集：

In [40]:
df = pd.read_csv('./data/company.csv')
df.head(3)

,EmployeeID,birthdate_key,age,city_name,department,job_title,gender
0,1318,1/3/1954,61,Vancouver,Executive,CEO,M
1,1319,1/3/1957,58,Vancouver,Executive,VP Stores,F
2,1320,1/2/1955,60,Vancouver,Executive,Legal Counsel,F


1. 分别只使用`query`和`loc`选出年龄不超过四十岁且工作部门为`Dairy`或`Bakery`的男性。

In [41]:
# df.query("age<=40 & (department == ['Dairy','Bakery']) & gender=='M'").head()
df.query("age<=40 & department in ['Dairy','Bakery'] & gender=='M'").head()

,EmployeeID,birthdate_key,age,city_name,department,job_title,gender
3611,5791,1/14/1975,40,Kelowna,Dairy,Dairy Person,M
3613,5793,1/22/1975,40,Richmond,Bakery,Baker,M
3615,5795,1/30/1975,40,Nanaimo,Dairy,Dairy Person,M
3617,5797,2/3/1975,40,Nanaimo,Dairy,Dairy Person,M
3618,5798,2/4/1975,40,Surrey,Dairy,Dairy Person,M


In [42]:
df.loc[(df.age<=40) & (df.department.isin(['Dairy','Bakery'])) & (df.gender=='M')].head()

,EmployeeID,birthdate_key,age,city_name,department,job_title,gender
3611,5791,1/14/1975,40,Kelowna,Dairy,Dairy Person,M
3613,5793,1/22/1975,40,Richmond,Bakery,Baker,M
3615,5795,1/30/1975,40,Nanaimo,Dairy,Dairy Person,M
3617,5797,2/3/1975,40,Nanaimo,Dairy,Dairy Person,M
3618,5798,2/4/1975,40,Surrey,Dairy,Dairy Person,M


2. 选出员工`ID`号 为奇数所在行的第1、第3和倒数第2列。

In [43]:
sum(df.duplicated('EmployeeID',keep=False))
df.iloc[(df.EmployeeID%2!=0).values, [0,2,-2]]

,EmployeeID,age,job_title
1,1319,58,VP Stores
3,1321,56,VP Human Resources
5,1323,53,"Exec Assistant, VP Stores"
6,1325,51,"Exec Assistant, Legal Counsel"
8,1329,48,Store Manager
...,...,...,...
6276,7659,26,Cashier
6277,7741,25,Cashier
6278,7801,25,Dairy Person
6280,8181,22,Cashier


3. 按照以下步骤进行索引操作：

* 把后三列设为索引后交换内外两层
* 恢复中间一层
* 修改外层索引名为`Gender`
* 用下划线合并两层行索引
* 把行索引拆分为原状态
* 修改索引名为原表名称
* 恢复默认索引并将列保持为原表的相对位置

In [44]:
df_new=df.set_index(list(df.columns[-3:])).swaplevel(0,2,axis=0)
df_new.head(3)

EmployeeID birthdate_key  age  city_name
gender job_title     department                                          
M      CEO           Executive         1318      1/3/1954   61  Vancouver
F      VP Stores     Executive         1319      1/3/1957   58  Vancouver
       Legal Counsel Executive         1320      1/2/1955   60  Vancouver

In [47]:
df_new=df_new.reset_index(['job_title'])
df_new.head(3)

job_title  EmployeeID birthdate_key  age  city_name
Gender department                                                         
M      Executive             CEO        1318      1/3/1954   61  Vancouver
F      Executive       VP Stores        1319      1/3/1957   58  Vancouver
       Executive   Legal Counsel        1320      1/2/1955   60  Vancouver

In [48]:
df_new=df_new.rename_axis(index={'gender':'Gender'})
df_new.head()

job_title  EmployeeID birthdate_key  age  \
Gender department                                                      
M      Executive                  CEO        1318      1/3/1954   61   
F      Executive            VP Stores        1319      1/3/1957   58   
       Executive        Legal Counsel        1320      1/2/1955   60   
M      Executive   VP Human Resources        1321      1/2/1959   56   
       Executive           VP Finance        1322      1/9/1958   57   

                   city_name  
Gender department             
M      Executive   Vancouver  
F      Executive   Vancouver  
       Executive   Vancouver  
M      Executive   Vancouver  
       Executive   Vancouver

In [49]:
df_new.index=df_new.index.map(lambda x:(x[0]+'-'+x[1]))
df_new

,job_title,EmployeeID,birthdate_key,age,city_name
M-Executive,CEO,1318,1/3/1954,61,Vancouver
F-Executive,VP Stores,1319,1/3/1957,58,Vancouver
F-Executive,Legal Counsel,1320,1/2/1955,60,Vancouver
M-Executive,VP Human Resources,1321,1/2/1959,56,Vancouver
M-Executive,VP Finance,1322,1/9/1958,57,Vancouver
...,...,...,...,...,...
F-Customer Service,Cashier,8036,8/9/1992,23,New Westminister
M-Customer Service,Cashier,8181,9/26/1993,22,Prince George
M-Customer Service,Cashier,8223,2/11/1994,21,Trail
F-Customer Service,Cashier,8226,2/16/1994,21,Victoria


In [55]:
df_new.index=df_new.index.map(lambda x:tuple(x.split('-')))
df_new


job_title  EmployeeID birthdate_key  age  \
M Executive                        CEO        1318      1/3/1954   61   
F Executive                  VP Stores        1319      1/3/1957   58   
  Executive              Legal Counsel        1320      1/2/1955   60   
M Executive         VP Human Resources        1321      1/2/1959   56   
  Executive                 VP Finance        1322      1/9/1958   57   
...                                ...         ...           ...  ...   
F Customer Service             Cashier        8036      8/9/1992   23   
M Customer Service             Cashier        8181     9/26/1993   22   
  Customer Service             Cashier        8223     2/11/1994   21   
F Customer Service             Cashier        8226     2/16/1994   21   
  Customer Service             Cashier        8264     6/13/1994   21   

                           city_name  
M Executive                Vancouver  
F Executive                Vancouver  
  Executive                Vancouver  
M Executive                Vancouver  
  Executive                Vancouver  
...                              ...  
F Customer Service  New Westminister  
M Customer Service     Prince George  
  Customer Service             Trail  
F Customer Service          Victoria  
  Customer Service         Vancouver  

[6284 rows x 5 columns]

In [59]:
df_new=df_new.rename_axis(index=('gender', 'department'))
df_new.reset_index().reindex(df.columns,axis=1)

,EmployeeID,birthdate_key,age,city_name,department,job_title,gender
0,1318,1/3/1954,61,Vancouver,Executive,CEO,M
1,1319,1/3/1957,58,Vancouver,Executive,VP Stores,F
2,1320,1/2/1955,60,Vancouver,Executive,Legal Counsel,F
3,1321,1/2/1959,56,Vancouver,Executive,VP Human Resources,M
4,1322,1/9/1958,57,Vancouver,Executive,VP Finance,M
...,...,...,...,...,...,...,...
6279,8036,8/9/1992,23,New Westminister,Customer Service,Cashier,F
6280,8181,9/26/1993,22,Prince George,Customer Service,Cashier,M
6281,8223,2/11/1994,21,Trail,Customer Service,Cashier,M
6282,8226,2/16/1994,21,Victoria,Customer Service,Cashier,F


### Ex2：巧克力数据集
现有一份关于巧克力评价的数据集：

In [23]:
df = pd.read_csv('./data/chocolate.csv')
df.head(3)

,Company,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating
0,A. Morin,2016,63%,France,3.75
1,A. Morin,2015,70%,France,2.75
2,A. Morin,2015,70%,France,3.00


1. 把列索引名中的`\n`替换为空格。

In [24]:
df.columns = [' '.join(i.split('\n')) for i in df.columns]
df.head()

,Company,Review Date,Cocoa Percent,Company Location,Rating
0,A. Morin,2016,63%,France,3.75
1,A. Morin,2015,70%,France,2.75
2,A. Morin,2015,70%,France,3.00
3,A. Morin,2015,70%,France,3.50
4,A. Morin,2015,70%,France,3.50


2. 巧克力`Rating`评分为1至5，每0.25分一档，请选出2.75分及以下且可可含量`Cocoa Percent`高于中位数的样本。

In [25]:
df['Cocoa Percent'] = df['Cocoa Percent'].apply(lambda x:float(x[:-1])/100)
df.query('(Rating<3)&(`Cocoa Percent`>`Cocoa Percent`.median())').head()

,Company,Review Date,Cocoa Percent,Company Location,Rating
33,Akesson's (Pralus),2010,0.75,Switzerland,2.75
34,Akesson's (Pralus),2010,0.75,Switzerland,2.75
36,Alain Ducasse,2014,0.75,France,2.75
38,Alain Ducasse,2013,0.75,France,2.50
39,Alain Ducasse,2013,0.75,France,2.50


3. 将`Review Date`和`Company Location`设为索引后，选出`Review Date`在2012年之后且`Company Location`不属于`France, Canada, Amsterdam, Belgium`的样本。

In [27]:
idx = pd.IndexSlice
exclude = ['France', 'Canada', 'Amsterdam', 'Belgium']
res = df.set_index(['Review Date', 'Company Location']).sort_index(level=0)
res.loc[idx[2012:,~res.index.get_level_values(1).isin(exclude)],:].head()

Company  Cocoa Percent  Rating
Review Date Company Location                                    
2012        Australia         Bahen & Co.           0.70    3.00
            Australia         Bahen & Co.           0.70    2.50
            Australia         Bahen & Co.           0.70    2.50
            Australia              Cravve           0.75    3.25
            Australia              Cravve           0.65    3.25